Let's try some basic tests of `VB.jl`:

In [1]:
include("VB.jl")
using VB
using Distributions

# Make a node

In [2]:
μ = rand(5, 5)
σ = rand(5, 5)
aa = RandomNode(:a, [:i, :j], Normal, μ, σ)
bb = RandomNode(:b, [:j, :k], Gamma, rand(5, 3), rand(5, 3))

VB.RandomNode{Distributions.Gamma}(:b,[:j,:k],5x3 Array{Distributions.Gamma,2}:
 Distributions.Gamma(α=0.2341186135210911, θ=0.9649351422758639)    …  Distributions.Gamma(α=0.728473678139731, θ=0.20010813276703399) 
 Distributions.Gamma(α=0.41560934007418204, θ=0.54016709350839)        Distributions.Gamma(α=0.3203071526069097, θ=0.19353926641853625)
 Distributions.Gamma(α=0.6902259590274851, θ=0.35684467900676764)      Distributions.Gamma(α=0.8366446754218231, θ=0.31033518271365734)
 Distributions.Gamma(α=0.6592764108950029, θ=0.6195766435991592)       Distributions.Gamma(α=0.26608819055600175, θ=0.364990448570903) 
 Distributions.Gamma(α=0.33551478040407545, θ=0.05088839846332349)     Distributions.Gamma(α=0.6151885476213583, θ=0.7675949926064045) )

In [3]:
cc = ConstantNode(:c, [:i, :j], rand(3, 4))

VB.ConstantNode{Float64}(:c,[:i,:j],3x4 Array{Float64,2}:
 0.243325   0.0772761  0.241217   0.957576
 0.0367407  0.365397   0.0699679  0.272531
 0.197855   0.216555   0.461437   0.564383)

In [4]:
cc = ConstantNode(rand(3, 4), [:i, :j])

VB.ConstantNode{Float64}(symbol("##const#7733"),[:i,:j],3x4 Array{Float64,2}:
 0.878628  0.873428  0.766667  0.362   
 0.239706  0.165815  0.590161  0.304251
 0.296727  0.93509   0.903521  0.971144)

In [5]:
nodes = Node[aa, bb]
fi = get_structure(nodes...)
fi

VB.FactorInds([:i,:j,:k],[5,3,5],Dict(:a=>[1,2],:b=>[2,3]))

In [6]:
a[i, j] ~ Normal(μ, σ)

VB.RandomNode{Distributions.Normal}(:a,[:i,:j],5x5 Array{Distributions.Normal,2}:
 Distributions.Normal(μ=0.1185012991601706, σ=0.90490595532671)       …  Distributions.Normal(μ=0.029945924375680333, σ=0.2688991851605318)
 Distributions.Normal(μ=0.94911665449286, σ=0.4163768154900689)          Distributions.Normal(μ=0.22862114650527077, σ=0.8117504427017941) 
 Distributions.Normal(μ=0.46997490469945813, σ=0.8348782760294375)       Distributions.Normal(μ=0.09273785326034156, σ=0.8792796596573758) 
 Distributions.Normal(μ=0.058602979077367445, σ=0.24822385690532078)     Distributions.Normal(μ=0.5899085840114116, σ=0.9514491545264085)  
 Distributions.Normal(μ=0.499322999672686, σ=0.7042926545486448)         Distributions.Normal(μ=0.8589886667315245, σ=0.7996934773990938)  )

In [7]:
c[i, j] ~ Const(rand(3, 4))

VB.ConstantNode{Float64}(:c,[:i,:j],3x4 Array{Float64,2}:
 0.382011  0.675517   0.0783089  0.29593  
 0.14824   0.839248   0.81254    0.0167539
 0.2083    0.0787781  0.380514   0.180021 )

# Make a factor

In [8]:
dims = (10, 2)

μ[j] ~ Normal(rand(dims[2]), ones(dims[2]))
τ ~ Gamma(1, 1)
x[i, j] ~ Normal(rand(dims), ones(dims))

f = @factor LogNormalFactor x μ τ;

# Make a (generated) value function

In [9]:
value(f)

-1.7570090928522661

In [10]:
g = @factor EntropyFactor x
value(g)

1.4189385332046727

# Let's make a simple model

We want a simple model of inference for a normal distribution:

$$
\begin{aligned}
    y \sim \mathcal{N}(\mu, \tau) \\
    \mu \sim \mathcal{N}(\mu_0, \tau_0) \\
    \tau \sim \mathrm{Ga}(\alpha, \beta) \\
    q(\mu) = \mathcal{N}(m, t) \\
    q(\tau) = \mathrm{Ga}(a, b)
\end{aligned}
$$

This model is conjugate, and can be solved by straightforward updates of the natural parameters of the posterior.

## Make the nodes

We need to make nodes for all the random variables that will need to be updated (we can also create nodes for parameter arrays, but these will be converted automatically if we don't).

In [11]:
dims = (20, 6)

# note: it won't matter much how we initialize here
μ[j] ~ Normal(zeros(dims[2]), ones(dims[2]))
τ[j] ~ Gamma(1.1 * ones(dims[2]), ones(dims[2]))
μ0[j] ~ Const(zeros(dims[2]))
τ0[j] ~ Const(2 * ones(dims[2]))
a0[j] ~ Const(1.1 * ones(dims[2]))
b0[j] ~ Const(ones(dims[2]))

y[i, j] ~ Const(rand(dims));

Now make factors: We need a Normal factor for the observation model plus a prior and an entropy for each node.

In [12]:
obs = @factor LogNormalFactor y μ τ
μ_prior = @factor LogNormalFactor μ μ0 τ0
τ_prior = @factor LogGammaFactor τ a0 b0

VB.LogGammaFactor{1}(VB.FactorInds([:j],[6],Dict(:τ=>[1],:b0=>[1],:a0=>[1])),VB.RandomNode{Distributions.Gamma}(:τ,[:j],[Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0)]),VB.ConstantNode{Float64}(:a0,[:j],[1.1,1.1,1.1,1.1,1.1,1.1]),VB.ConstantNode{Float64}(:b0,[:j],[1.0,1.0,1.0,1.0,1.0,1.0]))

In [13]:
value(obs), value(μ_prior), value(τ_prior)

(-1.344692382811822,-2.2655121234846454,0.007496947218732133)

In [22]:
m = VBModel(Dict(:μ => μ, :τ => τ), [obs, μ_prior, τ_prior])

VB.VBModel(Dict{Symbol,VB.Node}(:τ=>VB.RandomNode{Distributions.Gamma,2}(5x6 Array{Distributions.Gamma,2}:
 Distributions.Gamma(α=1.0, θ=1.0)  …  Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0),Dict{VB.Factor,Symbol}(VB.LogGammaFactor(VB.RandomNode{Distributions.Gamma,2}(#= circular reference =#),VB.ConstantNode{Float64,2}(5x6 Array{Float64,2}:
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1,Dict{VB.Factor,Symbol}()),VB.ConstantNode{Float64,2}(5x6 Array{Float64,2}:
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0,

In [23]:
m.nodes[:μ].factormap

Dict{VB.Factor,Symbol} with 2 entries:
  VB.LogNormalFactor(VB.C… => :μ
  VB.LogNormalFactor(VB.R… => :x

In [24]:
naturals(obs, Val{:x}, μ)

(
5x6 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0,

5x6 Array{Float64,2}:
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5)

In [25]:
naturals(obs, Val{:μ}, μ)

(
5x6 Array{Float64,2}:
 0.019526  0.64192   0.920968   0.221961  0.497111  0.376147
 0.645838  0.886014  0.635094   0.86881   0.173711  0.234695
 0.571483  0.605633  0.240604   0.252273  0.424832  0.433595
 0.246642  0.580497  0.81434    0.503185  0.399902  0.499845
 0.8564    0.315123  0.0795404  0.555742  0.340429  0.753323,

5x6 Array{Float64,2}:
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5)

In [26]:
naturals(obs, Val{:τ}, τ)

(0.5,
5x6 Array{Float64,2}:
 0.500191  0.706031  0.924091  0.524633  0.62356   0.570743
 0.708553  0.89251   0.701672  0.877416  0.515088  0.527541
 0.663296  0.683396  0.528945  0.531821  0.590241  0.594002
 0.530416  0.668488  0.831575  0.626598  0.579961  0.624922
 0.86671   0.549651  0.503163  0.654425  0.557946  0.783748)

In [27]:
method_exists(naturals, (LogNormalFactor, Type{Val{:x}}, RandomNode{Normal}))

true

In [28]:
check_conjugate(m.nodes[:τ])

true

In [29]:
μ.data

5x6 Array{Distributions.Normal,2}:
 Distributions.Normal(μ=0.0, σ=1.0)  …  Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)     Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)     Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)     Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)     Distributions.Normal(μ=0.0, σ=1.0)

In [30]:
update!(μ, Val{:conjugate})

In [31]:
μ.data

5x6 Array{Distributions.Normal,2}:
 Distributions.Normal(μ=-0.33974642587876164, σ=0.7071067811865476)  …  Distributions.Normal(μ=-0.08757735964789944, σ=0.7071067811865476)   
 Distributions.Normal(μ=0.1031230083822061, σ=0.7071067811865476)       Distributions.Normal(μ=-0.18759892024976002, σ=0.7071067811865476)   
 Distributions.Normal(μ=0.05054604661844582, σ=0.7071067811865476)      Distributions.Normal(μ=-0.046955363717725414, σ=0.7071067811865476)  
 Distributions.Normal(μ=-0.17915149839257358, σ=0.7071067811865476)     Distributions.Normal(μ=-0.00010971345274074199, σ=0.7071067811865476)
 Distributions.Normal(μ=0.2520128176967853, σ=0.7071067811865476)       Distributions.Normal(μ=0.17912620274862429, σ=0.7071067811865476)    